# Theoretischer Hintergrund

Was ist gerade der neueste Technologie Trend? Wie wird ein bestimmtes Produkt bewertet? Wie beurteilt die Gesellschaft aktuelle politische Themen? Sind Katzen Bilder immer noch süß oder nerven sie doch eher? <br>
All diese und noch viele weitere spannende Fragen lassen sich durch ein Untergebiet des **Text Minings**, der **Sentimentanalyse** beantworten.<br>
Wenn aus Texten mit statistischen und linguistischen Mitteln die relevanten Kerninformationen herausgefiltert werden, wird das Text Mining genannt.<br>
Das Wort *Sentiment* stammt aus dem Französischen, und bedeutet soviel wie Gefühl oder Empfindung. Dem Zufolge wird bei der Sentimentanalyse also eine Empfindung gegenüber einer bestimmten Sache untersucht. 


### Zweck der Sentimentanalyse
Die Sentimentanalyse ermittelt Stimmungen in sozialen Medien, bezogen auf Produkte, Kampagnen und Serviceleistungen. Dies gibt beispielsweise Unternehmen die Chance die Ursache von schlechten Meinungen und negativem Feedback zu analysieren und angemessen darauf zu reagieren. Des Weiteren könnnen sich Produkthersteller oder Dienstleistungsunternehmen einen Marktvorsprung dadurch generieren, dass sie bereits zukünftige Trends vor allen anderen erfahren.


### Beispiel
Ein Softwareunternehmen könnte beispielsweise diverse online Quellen, bei denen es hauptsächlich um Tech-Themen geht, anbinden, um von dort Daten abzuziehen. Diese Daten können dann genutzt werden, um herauszufinden, über was potenzielle Kunden aktuell diskutieren, welche Trends im Kommen sind und welche schon wieder am Verschwinden sind. So können sich Unternehmen noch besser den Bedürnissen Ihrer Kunden anpassen.

### Wie funktioniert die Sentimentanalyse
Bei der Sentimentanalyse werden Texte entweder *positiv,negativ* oder *neutral* klassifiziert.<br>
Um diese Einteilung erzeugen zu können ist einige Vorarbeit nötig, wie zum Beispiel das Entfernen von Hyperlinks und Satzzeichen.
Python stellt die Packages **NLTK** und **TextBlob** zur Verfügung, mit deren Hilfe ein Classifier implementiert werden kann. <br>Dieser lädt die Daten zunächst in einen Dataframe, um anschließend *Stopwords* zu entfernen und alle Buchstaben in Kleinbuchstaben zu konvertieren. Das Ergebnis ist ein gesäuberter Text. <br>Mit Hilfe der TextBlob Funktion kann die **Polarität** gemessen werden. Die Polarität ist ein Wert im Bereich zwischen $[-1.0,1.0]$ , wobei für Werte W folgendes gilt:<br> $ wi > 0 =    +(positiv) $<br> und<br>  $ wi < 0   =   - (negativ) $ <br>


### Was kann analysiert werden?
Die Sentimentanalyse kann in folgenden Bereichen angewendet werden:

* Produktbewertungen
* Service-/Dienstleistungsbewertungen
* Kundengespräche
* Trainingsdaten für maschinelles Lernen
* Videos
* Podcasts
* Bilder und Grafiken
* Soziale Medien:
    * Tweets
    * Facebookeinträge
    * Blogkommentare
    * Foreneinträge



# Aber wie kommen wir nun an die Daten?
Für das folgende Beispiel werden Twitterdaten verwendet. Diese wurden über die Twitter Streaming API gepulled.
Um diese nutzen zu können, muss ein [API Key](https://developer.twitter.com/en.html) angefordert werden.
Um auf die Twitter API zugreifen zu können, kann die Python Library [Tweepy](http://www.tweepy.org/) genutzt werden. <br>
Diese stellt Funktionen zur Verfügung, durch welche der Datenabzug parameterisiert und dadurch vereinfacht werden kann.

### Datenspeicherung
Um die Daten zu speichern bietet sich die dokumentenbasierte NoSQL Datenbank **MongoDB** an, da diese schnell zu installieren und einfach zu bedienen ist. <br> Diese kann kostenfrei auf [mongodb.com](https://www.mongodb.com/download-center#community) heruntergeladen werden. Hauptsächlich wird die MongoDB über die Konsole gesteuert, es gibt aber auch die Möglichkeit mit einer GUI zu arbeiten. Hierfür bietet sich der [Robomongo](https://robomongo.org/download) an.<br>
Python bietet das Package **Pymongo** an, um direkt mit der MongoDB zu kommunizieren. Das vereinfacht die Datenanalyse enorm, da nicht immer csv Dateien generiert und eingelesen werden müssen, sondern aktuelle Daten direkt aus der Datenbank extrahiert werden können.

### Datenbankverbindung
Nachdem alle Vorbereitungen getroffen wurden, geht es nun an den Datenabzug. Um uns mit der MongoDB verbinden zu können, benötigen wir den MongoClient aus dem Package Pymongo. Um die Twitter API zu nutzen benötigen wir das Package Tweepy.

In [1]:
import json
from pymongo import MongoClient
import tweepy
from tweepy.streaming import StreamListener

 Die folgende Tweepy Klasse wurde implementiert um englischsprachige Tweets mit dem Tag **'Android'** abzuziehen.

In [ ]:
# Connection to mongoDB
MONGO_HOST= 'mongodb://localhost:27017/DataScience'
# words to search for within the tweets
WORDS = ['#android' ]

# authentication
CONSUMER_KEY = "INSERT_CONSUMER_KEY"
CONSUMER_SECRET = "INSERT_CONSUMER_SECRET"
ACCESS_TOKEN = "INSERT_ACCESS_TOKEN"
ACCESS_TOKEN_SECRET = "INSERT_TOKEN_SECRET"

#This is a class provided by tweepy to access the Twitter Streaming API. 
class StreamListener(tweepy.StreamListener):    

    # Called initially to connect to the Streaming API
    def on_connect(self):
    
        print("You are now connected to the streaming API.")
    # On error - if an error occurs, display the error / status code
    def on_error(self, status_code):
    
        print('An Error has occured: ' + repr(status_code))
        return False
    #connect to mongoDB and store the tweets
    def on_data(self, data):
   
        try:
            client = MongoClient(MONGO_HOST)
            
            # Use DataScience Database
            db = client.DataScience
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweets to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
            #insert the data into the mongoDB into a collection called Android
            db.Android.insert_many(datajson)
        except Exception as e: print(e)
#Authentification via consumer key
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
#Filter tweets with defined tags and in english language
#streamer.filter(track=WORDS,languages =["en"])

Das ausgeführte Programm gibt folgendes aus:
![tweepy](img/tweepy.png)

Die Daten werden gestreamed und in der zuvor definierten Datenbank gespeichert:
<br>

<img src="img/mongodb.png" alt="mongodb" style="width: 600px",style="align:left";/>

Um die Sentimentanalyse durchführen zu können, müssen weitere Packages importiert werden.

In [1]:
import pandas as pd
from textblob import TextBlob
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

Anschließend wird die Verbindung zur MongoDB hergestellt.

In [7]:
#connection to MongoDB
CLIENT = MongoClient('localhost', 27017)
db = CLIENT['DataScience']

Bevor der Classifier zum Einsatz kommt, müssen die Tweets noch gecleaned werden.

In [8]:
#definition of stopwords and the vocabulary
from nltk.corpus import stopwords
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
stopWords = set(stopwords.words('english'))
print(stopWords)

{'all', 'herself', 'was', 'here', 'then', 'have', 'or', 'this', 'any', 'own', "isn't", 'than', 't', 'for', 'll', 'i', 'm', 'yours', 'some', "she's", 'them', 'whom', "haven't", 'with', "you'd", "wouldn't", 'a', 'as', 'hasn', 'up', 'from', "don't", 'don', 'themselves', 'into', 'too', 'few', 'but', 'they', 'down', 'were', 'should', 'shan', "aren't", 'at', 'because', 'you', 'wouldn', 'doing', "that'll", "mightn't", 'of', 've', 'we', 'he', 'theirs', 'both', 'only', 'between', 'did', 'when', 'above', 'that', 'very', "should've", 'who', 'what', 'shouldn', 'by', 'now', 'in', "it's", 'out', 'nor', 'each', 'how', 'is', 'after', 'so', "shouldn't", 'below', 'myself', 'ourselves', 'yourself', 'her', 'until', 'o', 'most', 'same', 'against', 'hadn', 'over', 'under', 'an', 'ours', 'if', "didn't", 'me', "hadn't", 'been', 'can', 'during', 'their', 'before', 'am', 'on', 'it', 'has', 'further', 'himself', 'such', 'does', "you've", 'just', "you're", 'be', 'weren', 'ma', "won't", 'mightn', 'him', 'hers', 'c

Das folgende Modul berechnet die Polarität für jeden Tweet und klassifiert diese so zu bestimmten Sentimenten. <br> Die Tweets erhalten alle ein Label bezüglich ihres Sentiments:<br>
* 1 für positiv
* 0 für neutral
* -1 für negativ
<br> 

Anschließend wird die Collection geupdated. Dies geschieht indem die neuen Labels den Datenbankeinträgen hinzugefügt werden.

In [1]:
class SentimentClassifier():
    """Classifier for the sentiment of documents"""
    
    
    def make_df(self, collection, parameter):
        """Loads the data form the given collection
        
        Args: 
            collection(pymongo.collection): Collection from which the data is loaded
	        parameter(str): Name of the field in which the text is stored
        
       
        Returns: 
            pandas.dataframe: Data frame of the loaded data 
                
        """
        df = pd.DataFrame(list(collection.find({"Sentiment":{"$exists": False}},{parameter:1, "_id": 0})))
        print("data is loaded: "+str(len(df)))
        return df
    def load_text(self,df,parameter):
        """loads text out of a dataframe
        
        Args:   
	        df(pandas.dataframe): dataframe containing the data 
	        parameter(str): name of the field where the text is stored
        
        Returns: 
            List of the texts stored in the data frame
        """
        return df[parameter].tolist()
    def update(self, collection, text, name, data, source):
        """Updates a document
        
        Args:
            collection(pymongo.collection): collection, that is updated
	        text(str): text to identify the document
	        name(str): field name of the stored value
	        data(no specific datatype): value to be stored
               
        Returns: 
            None
        """
        collection.update_one({source : text}, {'$set': {name: data}})
    def process_text(self, text):
        """Preprocesses a given text
        
        Args: 
            text(str): text, that is preprocessed
        
        Returns: 
            tokens: tokens of a text 
        
        """
        if text.startswith('@null'):
            return "[Text not available]"
        # Remove tickers
        text = re.sub(r'\$\w*','',text)
        # Remove hyperlinks
        text = re.sub(r'https?:\/\/.*\/\w*','',text)
        # Remove puncutations like "'s"
        text = re.sub(r'['+string.punctuation+']+', ' ',text) 
        twtok = TweetTokenizer(strip_handles=True, reduce_len=True)
        tokens = twtok.tokenize(text)
        #if i not in stopwords and len(i) > 2 and i in english_vocab]
        tokens = [i.lower() for i in tokens] 
        return tokens
    def clean_texts(self, df,parameter):
        """cleans the texts of a data frame and stores the cleaned texts
        
        Args:
            df(pandas.dataframe): dataframe which texts are cleaned
	        paramter(str): name of the field where the text is stored
           
        Returns:
            list: List of the cleaned texts
            list: list of the original texts 
        """
        texts = self.load_text(df,parameter)
        # store all clean words 
        words = []
        for t in texts:
            words += self.process_text(t)
            
        #store the cleaned texts
        cleaned_texts = []
        for text in texts:
            words = self.process_text(text)
            #Form sentences of processed words
            cleaned_text = " ".join(w for w in words if len(w) > 2 and w.isalpha())
            cleaned_texts.append(cleaned_text)
        df['CleanText'] = cleaned_texts
        return cleaned_texts, texts
    def analyse_sentiment(self,text):
        """Classifies the sentiment of a text
        
        Args: 
            text(str): text, that is classified
        
        Returns: 
            int: Sentiment, -1 for negative, 0 for neutral, 1 for positive
        
        """
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1
    def update_sentiment(self, collection, parameter, df):
        """Analyses the sentiment and stores the results for the documents in a collection
        
        Args: 
            collection(pymongo.collection): collection, that is analyzed
	        paramter(str): name of the fields where the text is stored
	        df(pandas.dataframe): dataframe of the collection
        
        
        Returns: 
            None
        
        """
        cleaned_texts, texts = self.clean_texts(df, parameter)
        sentimented_texts = []
        counter = 0
        print("Hooray data is now cleaned")
        for tw in cleaned_texts:
            #analysis the sentiment
            sentiment = self.analyse_sentiment(tw) 
            sentimented_texts.append(sentiment)
            #update of sentimented texts
            self.update(collection, texts[counter], 'Sentiment', sentiment, parameter) 
            counter +=1
        df['Sentiment'] = sentimented_texts
    
        print("Data is classified according to sentiments")
        pos_texts = [ text for index, text in enumerate(df["CleanText"]) if df['Sentiment'][index] > 0]
        neu_texts = [ text for index, text in enumerate(df["CleanText"]) if df['Sentiment'][index] == 0]
        neg_texts = [ text for index, text in enumerate(df["CleanText"]) if df['Sentiment'][index] < 0]
    
        print("Percentage of positive tweets: {}%".format(len(pos_texts)*100/len(df[parameter])))
        print("Percentage of neutral tweets: {}%".format(len(neu_texts)*100/len(df[parameter])))
        print("Percentage of negative tweets: {}%".format(len(neg_texts)*100/len(df[parameter])))

if __name__ == "__main__":
    #classify the sentiment of Twitter
    c = SentimentClassifier()   
    #p = "text"
    #collection = db["Android"]
    #df_T = c.make_df(collection,p)
    #tweets = c.load_text(df_T,p)
    #c.update_sentiment(collection, p, df_T)

Das Ergebnis zeigt zwar deutlich, dass *Android* offenbar als überwiegend positiv empfunden wird, <br>jedoch lässt die gleichzeit hohe Prozenzzahl von neutralen Postings die Vermutung zu, <br>
dass der Classifier noch besser trainiert werden müsste um ein noch genaueres Ergebnis zu bekommen.